In [214]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
%matplotlib inline

In [184]:
df = pd.read_csv('insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [185]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(df[['age', 'affordibility']], df['bought_insurance'], test_size=0.5)

In [186]:
train_x.head()

,age,affordibility
24,50,1
6,55,0
22,40,1
4,46,1
15,55,1


In [187]:
len(train_x)

14

In [188]:
len(test_x)

14

In [189]:
train_x_scaled = train_x.copy()
train_x_scaled['age'] = train_x['age'] / 100

test_x_scaled = test_x.copy()
test_x_scaled['age'] = test_x['age'] / 100

In [190]:
train_x_scaled.head()

,age,affordibility
24,0.50,1
6,0.55,0
22,0.40,1
4,0.46,1
15,0.55,1


In [191]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

In [192]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [193]:
model.fit(train_x_scaled, train_y, epochs=500)

Epoch 1/500
1/1 [==============================] - 0s 166ms/step - loss: 0.5814 - accuracy: 0.5714
Epoch 2/500
1/1 [==============================] - 0s 2ms/step - loss: 0.5811 - accuracy: 0.5714
Epoch 3/500
1/1 [==============================] - 0s 2ms/step - loss: 0.5809 - accuracy: 0.5714
Epoch 4/500
1/1 [==============================] - 0s 2ms/step - loss: 0.5807 - accuracy: 0.5714
Epoch 5/500
1/1 [==============================] - 0s 1ms/step - loss: 0.5805 - accuracy: 0.5714
Epoch 6/500
1/1 [==============================] - 0s 2ms/step - loss: 0.5803 - accuracy: 0.5714
Epoch 7/500
1/1 [==============================] - 0s 2ms/step - loss: 0.5801 - accuracy: 0.5714
Epoch 8/500
1/1 [==============================] - 0s 2ms/step - loss: 0.5799 - accuracy: 0.5714
Epoch 9/500
1/1 [==============================] - 0s 2ms/step - loss: 0.5797 - accuracy: 0.5714
Epoch 10/500
1/1 [==============================] - 0s 1ms/step - loss: 0.5795 - accuracy: 0.5714
Epoch 11/500
1/1 [=========

In [194]:
model.evaluate(test_x_scaled, test_y)

1/1 [==============================] - 0s 50ms/step - loss: 0.8251 - accuracy: 0.5000


[0.8250552415847778, 0.5]

In [195]:
model.predict(test_x_scaled)

1/1 [==============================] - 0s 20ms/step


array([[0.80277085],
       [0.7911805 ],
       [0.7989622 ],
       [0.4625597 ],
       [0.7654494 ],
       [0.7622183 ],
       [0.41409284],
       [0.42281026],
       [0.7941244 ],
       [0.42281026],
       [0.80559146],
       [0.7675865 ],
       [0.4744529 ],
       [0.7622183 ]], dtype=float32)

In [196]:
test_y

17    1
27    0
25    1
3     0
20    0
10    0
18    0
16    1
14    1
1     0
9     1
26    0
7     1
19    0
Name: bought_insurance, dtype: int64

In [197]:
coef, intercept = model.get_weights()

coef, intercept

(array([[0.5970577],
        [1.5179217]], dtype=float32),
 array([-0.46051204], dtype=float32))

In [198]:
def sigmoid(x):
    import math
    return 1 / (1 + math.exp(-x))


sigmoid(18)

0.9999999847700205

In [199]:
def prediction_function(age, affordability):
    coef, intercept = model.get_weights()
    return sigmoid(coef[0] * age + coef[1] * affordability + intercept)

In [200]:
prediction_function(0.47, 1)

0.7921652558869355

In [201]:
def log_loss(y_true, y_predicted):
    """
    Computes the log loss of the y_true and the y_predicted, aka the score of how wrong your model was. Another term for log loss is binary cross entropy

    Args:
        y_true (_type_): The source of truth for whether the predictions were correct
        y_predicted (_type_): The predictions of the y values to compare to the source of truth

    Returns:
        _type_: _description_
    """
    epsilon = 1e-15 # epsilon constant to round down the numbers
    y_predicted_new = [max(i, epsilon) for i in y_predicted] # takes each y predicted value, and pulls the max of it and epsilon
    y_predicted_new = [min(i, 1-epsilon) for i in y_predicted_new] # takes each value in y_predicted, and computes the minimum of itself or 1 - epsilon
    y_predicted_new = np.array(y_predicted_new) # converts y_predicted_new into a numpy array
    return -np.mean(y_true * np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new)) # implementation of the log loss function

def log_loss_singular(y_true_point, y_predicted_point):
    epsilon = 1e-15
    y_predicted_new = max(y_predicted_point, epsilon)
    y_predicted_new = min(y_predicted_new, 1 - epsilon)
    return -1 * (y_true_point * math.log(y_predicted_new) + (1 - y_true_point) * math.log(1 - y_predicted_new))

    

In [202]:
def sigmoid_numpy(x):
    return 1 / (1+np.exp(-x))


sigmoid_numpy(np.array([12, 0, 1]))

array([0.99999386, 0.5       , 0.73105858])

In [203]:
def gradient_descent(age, affordability, y_true, epochs, loss_threshold = 0.0):
    # w1, w2, bias
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)

    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias ## formula
        y_predicted = sigmoid_numpy(weighted_sum) # compute the sigmoid of the processed data
        loss = log_loss(y_true, y_predicted) # compute the loss, passing in the y_true and y_predicted into the log loss function

        w1d = (1/n) * np.dot(np.transpose(age),(y_predicted - y_true)) # compute the partial derivative of weight 1, which is a transposition of age, combined with the dot product of y_predicted - y_true
        w2d = (1/n) * (np.dot(np.transpose(affordability),(y_predicted - y_true))) # compute the partial derivative of weight 2, which is the transposition of age, combined with the dot product of y_predicted - y_true
        bias_d = np.mean(y_predicted - y_true) # compute the partial derivate of bias, which is the mean of y_predicted - y_true

        w1 = w1 - rate * w1d # compute the new weight 1, adhering to the formula
        w2 = w2 - rate * w2d # compute the new weight 2, adhering to the formula
        bias = bias - rate * bias_d # compute the new bias, adhering to the formula

        if (i % 50) == 0:
            print('Epoch:{}, w1:{}, w2:{}, bias:{}, loss:{}'.format(i, w1, w2, bias, loss)) # print out diagnostic information

        if loss <= loss_threshold: # compare if loss is less then loss threshold
            print('Epoch:{}, w1:{}, w2:{}, bias:{}, loss:{}'.format(i, w1, w2, bias, loss)) # print out diagnostic information
            break
    return w1, w2, bias # return the new weights, and bias


def stochastic_gradient_descent(age, affordability, y_true, epochs, loss_threshold = 0.0, w1, w2, bias):
    rate = 0.5
    n = len(age)

    random_ind = random.randint(0, len(age))

    for i in range(epochs):
        random_age = age[random_ind]
        random_affordability = affordability[random_ind]

        weighted_sum = w1 * random_age + w2 * random_affordability + bias
        y_predicted = sigmoid(weighted_sum)

        


In [204]:
gradient_descent(train_x['age'], train_x['affordibility'], train_y, 1000, 0.25)

Epoch:0, w1:-5.678571428483725, w2:0.9285714285751027, bias:-0.21428571428178156, loss:12.038484834159751
Epoch:50, w1:-3.103210888074347, w2:3.07452846326587, bias:-2.9106437763301787, loss:14.802675425288948
Epoch:100, w1:-1.0900596516611802, w2:5.2151416827728685, bias:-5.627203848012025, loss:14.802675425288948
Epoch:150, w1:0.7850599821066968, w2:7.3511062266861025, bias:-8.348397742540625, loss:14.802675425288948
Epoch:200, w1:2.729261251356811, w2:9.489806117761036, bias:-11.066837995722576, loss:14.802675425288948
Epoch:250, w1:4.669481237508886, w2:11.628648768402009, bias:-13.785113093735458, loss:14.802675425288948
Epoch:300, w1:6.6043169007664195, w2:13.767475885071418, bias:-16.50316904301854, loss:14.802675425288948
Epoch:350, w1:5.903554735788858, w2:15.849825370667286, bias:-19.27189728625454, loss:14.802675425288948
Epoch:355, w1:-1.049204060468038, w2:15.87138543515648, bias:-19.678912247875047, loss:0.197933471887865


(-1.049204060468038, 15.87138543515648, -19.678912247875047)

In [205]:
print(model.get_weights())

[array([[0.5970577],
       [1.5179217]], dtype=float32), array([-0.46051204], dtype=float32)]


In [206]:
class NeuralNetwork:

    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 1

    def fit_batch_gradient(self, x_data, y_data, epochs, loss_threshold):
       self.w1, self.w2, self.bias = gradient_descent(x_data['age'], x_data['affordibility'], y_data, epochs, loss_threshold)

    def fit_stochastic_gradient(self, x_data, y_data, epochs, loss_threshold):
        self.w1, self.w2, self.bias = stochastic_gradient_descent(x_data['age'], x_data['affordibility'], y_data, epochs, loss_threshold)

    def sigmoid(self, value):
        return 1 / (1 + np.exp(-value))

    def predict(self, x_data):
        age = x_data['age']
        affordability = x_data['affordibility']
        return self.sigmoid(self.w1 * age + self.w2 * affordability + self.bias)

    def test(self, x_data, y_true):
        predicted_y = self.predict(x_data)
        return np.mean([1 if predicted_y[i] == y_true[i] else 0 for i in range(len(predicted_y))])


In [207]:
test_y.head()

17    1
27    0
25    1
3     0
20    0
Name: bought_insurance, dtype: int64

In [208]:
nn = NeuralNetwork()

nn.fit(train_x_scaled, train_y, 500, 0.5)

nn.predict(test_x_scaled).head()

Epoch:0, w1:0.9829589502357541, w2:0.996609914284597, bias:-0.08688511859793165, loss:0.5813607990791463
Epoch:12, w1:0.9577914521087059, w2:1.2199498607772614, bias:-0.6856147079955446, loss:0.49736409111906354


17    0.748354
27    0.726099
25    0.741071
3     0.453246
20    0.676005
dtype: float64

In [209]:
model.get_weights()

[array([[0.5970577],
        [1.5179217]], dtype=float32),
 array([-0.46051204], dtype=float32)]

In [212]:
model.predict(test_x_scaled)[0:5]

1/1 [==============================] - 0s 11ms/step


array([[0.80277085],
       [0.7911805 ],
       [0.7989622 ],
       [0.4625597 ],
       [0.7654494 ]], dtype=float32)